In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from datetime import datetime, date
from time import mktime
import mysql.connector
import pymysql
from sqlalchemy import create_engine

In [2]:

#date to start query
start = date(2012, 1,2)
unix_start = str(int(mktime(start.timetuple()))) +'/'

#date to end query
end = pd.to_datetime('today')
unix_end = str(int(mktime(end.timetuple()))) 

#api url
url = 'https://coinmetrics.io/api/v1/get_asset_data_for_time_range/'

#coin ticker
coin = 'btc/'

#data to be retreived
data_type_price = 'price(usd)/'
data_type_mcap = 'marketcap(usd)/'

#api call
coin_data = requests.get( url + coin+'/' + data_type_mcap + str(unix_start) + str(unix_end))

In [3]:
coin_list = ['btc','eth', 'eos']
#pd.options.display.int_format = '{:.9f}'.format
new_df = pd.DataFrame(None)

#converting json object
'''data = coin_data.json()
data = pd.DataFrame(data)
data = pd.DataFrame(data['result'].values.tolist(), columns=['Date','Price'])'''

for i in coin_list:

        coin_prices = requests.get( url + i+ '/' + data_type_price + str(unix_start) + str(unix_end))
        price_data = coin_prices.json()
        
        coin_mcap = requests.get( url + i+ '/' + data_type_mcap + str(unix_start) + str(unix_end))
        mcap_data = coin_mcap.json()
        
        price_df = pd.DataFrame(price_data)
        mcap_df = pd.DataFrame(mcap_data)
        
        price_df = pd.DataFrame(price_df['result'].values.tolist(), columns=['date',i+'_price'])
        mcap_df = pd.DataFrame(mcap_df['result'].values.tolist(), columns=['date',i+'_market_cap'])
        
        
        if new_df.empty:
            new_df = price_df.merge(mcap_df, on ="date")
        else:
            new_df = pd.merge(new_df, price_df, on ="date", how='outer')
            new_df = pd.merge(new_df, mcap_df, on ="date", how='outer')
            
new_df['date'] = pd.to_datetime(new_df['date'],unit='s')
new_df = new_df.set_index('date')

In [4]:
#data = coin_data.json()
#data = pd.DataFrame(data)
#data = pd.DataFrame(data['result'].values.tolist(), columns=['Date','MarketCap'])
#data

In [5]:
new_df.tail()

,btc_price,btc_market_cap,eth_price,eth_market_cap,eos_price,eos_market_cap
date,,,,,,
2018-12-09,3473.23,6.048663e+10,92.04,9.544932e+09,1.84,1.669327e+09
2018-12-10,3612.05,6.290965e+10,94.99,9.852083e+09,2.02,1.828054e+09
2018-12-11,3497.55,6.092103e+10,91.58,9.500126e+09,1.93,1.747946e+09
2018-12-12,3421.46,5.960081e+10,88.61,9.194590e+09,1.85,1.676151e+09
2018-12-13,3487.88,6.076348e+10,90.66,9.408682e+09,1.98,1.795116e+09
